# **`04 FINE TUNNING`**

Cuando un nuevo persona ingrese, sera necesario añadir al modelo esos nuevos personas, para no entrenar otra vez todo, se le añade nuevas capas

In [ ]:
import numpy as np
import os
import cv2
import json
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
# Configuración
IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.0001  # Tasa de aprendizaje más baja

In [ ]:
# Función para cargar imágenes y etiquetas
def load_data(dir_path):
    images = []
    labels = []
    for person_name in os.listdir(dir_path):
        person_path = os.path.join(dir_path, person_name)
        if os.path.isdir(person_path):
            for img_name in os.listdir(person_path):
                img_path = os.path.join(person_path, img_name)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                img = img[:, :, ::-1] / 255.0  # BGR to RGB and normalize
                images.append(img)
                labels.append(person_name)
    return np.array(images), np.array(labels)

In [ ]:
from google.colab import drive
# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cargar el modelo existente
# model = load_model('/content/drive/MyDrive/Personales/Estudio/BigData/Mod11/TareaFinal/face_recognition_model.h5')
model = load_model('/content/drive/MyDrive/ProyectoFotos/Modelo/face_recognition_model.h5')

# Cargar las etiquetas existentes
# with open('/content/drive/MyDrive/Personales/Estudio/BigData/Mod11/TareaFinal/face_recognition_model.json', 'r') as f:
with open('/content/drive/MyDrive/ProyectoFotos/JSON/face_recognition_model.json', 'r') as f:
    existing_labels = json.load(f)

In [ ]:
# Cargar una parte de los datos originales (ajusta la ruta según sea necesario)
# original_dir_path = '/content/drive/MyDrive/Personales/Estudio/BigData/Mod11/TareaFinal/personas_224x224'
original_dir_path = '/content/drive/MyDrive/ProyectoFotos/Celebrity_Faces_Dataset/personas_nuevas_224x224'
original_images, original_labels = load_data(original_dir_path)

In [ ]:
# Cargar nuevos datos
# new_dir_path = '/content/drive/MyDrive/Personales/Estudio/BigData/Mod11/TareaFinal/personas_nuevas_224x224'
new_dir_path = '/content/drive/MyDrive/ProyectoFotos/Celebrity_Faces_Dataset/personas_nuevas_224x224'
new_images, new_labels = load_data(new_dir_path)

In [ ]:
# Combinar datos originales y nuevos
all_images = np.concatenate([original_images, new_images])
all_labels = np.concatenate([original_labels, new_labels])

In [ ]:
# Actualizar el mapeo de etiquetas
all_unique_labels = list(set(list(existing_labels.keys()) + list(np.unique(all_labels))))
le = LabelEncoder()
le.fit(all_unique_labels)
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

In [ ]:
# Codificar todas las etiquetas
all_labels_encoded = le.transform(all_labels)
all_labels_one_hot = to_categorical(all_labels_encoded, num_classes=len(label_mapping))

In [ ]:
# Modificar la capa de salida del modelo
model.pop()  # Eliminar la última capa
for layer in model.layers[:-2]:  # Congelar todas las capas excepto las dos últimas
    layer.trainable = False
new_output_layer = Dense(len(label_mapping), activation='softmax')
model.add(new_output_layer)

In [ ]:
# Compilar el modelo actualizado con una tasa de aprendizaje más baja
optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Fine-tuning
history = model.fit(all_images, all_labels_one_hot, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)

Epoch 1/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 10s 107ms/step - accuracy: 0.1938 - loss: 2.6296 - val_accuracy: 0.1514 - val_loss: 2.4153
Epoch 2/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - accuracy: 0.5602 - loss: 1.5674 - val_accuracy: 0.4159 - val_loss: 1.8029
Epoch 3/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 10s 67ms/step - accuracy: 0.6897 - loss: 1.2074 - val_accuracy: 0.5856 - val_loss: 1.4270
Epoch 4/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 11s 80ms/step - accuracy: 0.7845 - loss: 0.9696 - val_accuracy: 0.6774 - val_loss: 1.1979
Epoch 5/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 12s 103ms/step - accuracy: 0.8219 - loss: 0.8234 - val_accuracy: 0.6896 - val_loss: 1.1339
Epoch 6/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 8s 95ms/step - accuracy: 0.8543 - loss: 0.6917 - val_accuracy: 0.7584 - val_loss: 0.9596
Epoch 7/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 8s 102ms/step - accuracy: 0.8877 - loss: 0.5917 - val_accuracy: 0.7783 - val_loss: 0.8901
Epoch 8/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - accuracy: 0.9084 - loss: 0.5318 - val_accuracy: 0.8

In [ ]:
def save_labels(labels, filename):
    labels = {k: int(v) for k, v in labels.items()}
    with open(filename, 'w') as f:
        json.dump(labels, f)

In [ ]:
# Guardar el modelo y las etiquetas actualizadas
# model.save('/content/drive/MyDrive/Personales/Estudio/BigData/Mod11/TareaFinal/face_recognition_model_updated.h5')
model.save('/content/drive/MyDrive/ProyectoFotos/Modelo/face_recognition_model_updated.h5')

def save_labels(labels, filename):
    labels = {k: int(v) for k, v in labels.items()}
    with open(filename, 'w') as f:
        json.dump(labels, f)

# save_labels(label_mapping, '/content/drive/MyDrive/Personales/Estudio/BigData/Mod11/TareaFinal/face_recognition_model_updated.json')
save_labels(label_mapping, '/content/drive/MyDrive/ProyectoFotos/JSON/face_recognition_model_updated.json')

print("Modelo y etiquetas actualizados guardados exitosamente.")


Modelo y etiquetas actualizados guardados exitosamente.
